In [3]:
from prophet import Prophet
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import joblib

In [4]:
df = pd.read_csv('error_ratio.csv' )

In [5]:
df.head()

,ds,Error
0,2024-04-15 23:32:00,0.000128
1,2024-04-15 23:33:00,0.000128
2,2024-04-15 23:34:00,0.000000
3,2024-04-15 23:35:00,0.000000
4,2024-04-15 23:36:00,0.000000


In [6]:
ts=df[['ds','Error']]

In [7]:
ts['ds'] = pd.to_datetime(ts['ds'])

In [8]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43272 entries, 0 to 43271
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      43272 non-null  datetime64[ns]
 1   Error   43272 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 676.2 KB


In [9]:
Prophet

prophet.forecaster.Prophet

In [10]:
ts.rename(columns={"Error": "y"}, inplace=True)


In [11]:

model = Prophet(interval_width=0.99, weekly_seasonality=True, yearly_seasonality=False, daily_seasonality=True)
model.fit(ts)
forecast = model.predict(ts)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpcns0se8s/22x694lw.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcns0se8s/3vvxpj8d.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=14924', 'data', 'file=/tmp/tmpcns0se8s/22x694lw.json', 'init=/tmp/tmpcns0se8s/3vvxpj8d.json', 'output', 'file=/tmp/tmpcns0se8s/prophet_modelgbixpjl8/prophet_model-20240525231733.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
23:17:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:17:39 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [12]:
forecast.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2024-04-15 23:32:00,0.000364,-0.012240,0.013295,0.000364,0.000364,0.000096,0.000096,0.000096,0.000077,0.000077,0.000077,0.000019,0.000019,0.000019,0.0,0.0,0.0,0.000460
1,2024-04-15 23:33:00,0.000364,-0.013143,0.014843,0.000364,0.000364,0.000095,0.000095,0.000095,0.000076,0.000076,0.000076,0.000019,0.000019,0.000019,0.0,0.0,0.0,0.000458
2,2024-04-15 23:34:00,0.000364,-0.014187,0.015021,0.000364,0.000364,0.000094,0.000094,0.000094,0.000075,0.000075,0.000075,0.000019,0.000019,0.000019,0.0,0.0,0.0,0.000457
3,2024-04-15 23:35:00,0.000364,-0.012565,0.014583,0.000364,0.000364,0.000092,0.000092,0.000092,0.000073,0.000073,0.000073,0.000019,0.000019,0.000019,0.0,0.0,0.0,0.000456
4,2024-04-15 23:36:00,0.000363,-0.012831,0.015411,0.000363,0.000363,0.000091,0.000091,0.000091,0.000072,0.000072,0.000072,0.000019,0.000019,0.000019,0.0,0.0,0.0,0.000454


In [13]:
# Предикт делаем на полную исходную выборку:
performance = pd.merge(ts, forecast[['ds','yhat_lower', 'yhat',  'yhat_upper']], on='ds')
performance.head()


,ds,y,yhat_lower,yhat,yhat_upper
0,2024-04-15 23:32:00,0.000128,-0.012240,0.000460,0.013295
1,2024-04-15 23:33:00,0.000128,-0.013143,0.000458,0.014843
2,2024-04-15 23:34:00,0.000000,-0.014187,0.000457,0.015021
3,2024-04-15 23:35:00,0.000000,-0.012565,0.000456,0.014583
4,2024-04-15 23:36:00,0.000000,-0.012831,0.000454,0.015411


In [14]:
performance.rename(columns={'y': 'y_real', 'yhat_lower': 'y_pred_lower', 'yhat': 'y_pred', 'yhat_upper': 'y_pred_upper'}, inplace=True)
performance = performance[['ds', 'y_pred_lower', 'y_pred', 'y_pred_upper', 'y_real']]
performance.head()

,ds,y_pred_lower,y_pred,y_pred_upper,y_real
0,2024-04-15 23:32:00,-0.012240,0.000460,0.013295,0.000128
1,2024-04-15 23:33:00,-0.013143,0.000458,0.014843,0.000128
2,2024-04-15 23:34:00,-0.014187,0.000457,0.015021,0.000000
3,2024-04-15 23:35:00,-0.012565,0.000456,0.014583,0.000000
4,2024-04-15 23:36:00,-0.012831,0.000454,0.015411,0.000000


Регулируем через std порог детектирования

In [15]:
# Вычисляем стандартное отклонение предсказанных значений
std_dev = performance['y_pred'].std()

# Определяем аномалии с более строгим порогом
performance['anomaly'] = performance.apply(
    lambda row: 1 if (row['y_real'] < (row['y_pred_lower'] - 20 * std_dev)) or (row['y_real'] > (row['y_pred_upper'] + 20 * std_dev)) else 0, axis=1)

anomalies = performance[performance['anomaly'] == 1].sort_values(by='ds')
anomalies.head()

,ds,y_pred_lower,y_pred,y_pred_upper,y_real,anomaly
3642,2024-04-18 12:14:00,-0.013074,0.000472,0.013727,0.024476,1
3643,2024-04-18 12:15:00,-0.013545,0.000468,0.014846,0.023774,1
5775,2024-04-19 23:59:00,-0.013811,0.000359,0.014720,0.032563,1
5776,2024-04-20 00:00:00,-0.013191,0.000358,0.014727,0.032840,1
9266,2024-04-22 10:10:00,-0.011323,0.001494,0.015063,0.445168,1


In [16]:
performance['color'] = np.where(performance['anomaly']== 1, 'red', 'green')
performance['name'] = np.where(performance['anomaly'] == 1, 'Anomaly','Within Confidence Interval')

In [17]:
# Преобразуем исходные данные в DataFrame для Plotly
performance['timestamp'] = performance['ds']
performance['target'] = performance['y_real']
ts_df = performance[['timestamp', 'target']]

# Создаем график временного ряда
fig = px.line(ts_df, x='timestamp', y='target', title='Time Series with Anomalies')

# Добавляем аномалии на график
anomaly_points = performance[performance['anomaly'] == 1]
fig.add_scatter(x=anomaly_points['timestamp'], y=anomaly_points['target'],
                mode='markers', marker=dict(color='red', size=10), name='Anomalies')


In [18]:
# Подготовка итогового DataFrame
result = performance[['ds', 'y_real', 'anomaly']]
result.rename(columns={'ds': 'timestamp', 'y_real': 'Error'}, inplace=True)
result['model'] = 'Prophet'

result.rename(columns={'ds': 'timestamp', 'Error': 'value'}, inplace=True)
result['Модель'] = 'Prophet'
result['Метрика'] = 'Error'
result['is_anomaly']=result['anomaly']
result['is_anomaly'] = result['is_anomaly'].astype(bool)
# Сохраняем результат в CSV файл
result = result[['Модель', 'Метрика', 'timestamp', 'value',  'is_anomaly']]
result.to_csv('Prophet_Error.csv', index=False)

print("Результаты модели сохранены в файл 'result.csv'")

<ipython-input-18-84f76638ad30>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-84f76638ad30>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-84f76638ad30>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-18-84f76638ad30>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

Результаты модели сохранены в файл 'result.csv'


In [19]:
result

,Модель,Метрика,timestamp,value,is_anomaly
0,Prophet,Error,2024-04-15 23:32:00,0.000128,False
1,Prophet,Error,2024-04-15 23:33:00,0.000128,False
2,Prophet,Error,2024-04-15 23:34:00,0.000000,False
3,Prophet,Error,2024-04-15 23:35:00,0.000000,False
4,Prophet,Error,2024-04-15 23:36:00,0.000000,False
...,...,...,...,...,...
43267,Prophet,Error,2024-05-16 00:54:00,0.000000,False
43268,Prophet,Error,2024-05-16 00:55:00,0.000000,False
43269,Prophet,Error,2024-05-16 00:56:00,0.000206,False
43270,Prophet,Error,2024-05-16 00:57:00,0.000211,False
